In [1]:
import pymysql
import pandas as pd
from xgboost import plot_importance
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
import xgboost as xgb
print(xgb.__version__)

0.90


## 품질데이터 가져오기

In [3]:
def get_quality_data_list_for_predict():
    conn = pymysql.connect(host='127.0.0.1', user='root', password='carry789', db='projectdata', charset='utf8')

    sql = '''
        select product_key, product_size_l, product_size_w, product_size_h, product_test
        from product_quality
    '''

    cursor = conn.cursor()
    cursor.execute(sql)
    row = cursor.fetchall()

    data_list = []

    for obj in row:
        data_dic = {
            'product_key': obj[0],
            'product_size_l': obj[1],
            'product_size_w': obj[2],
            'product_size_h': obj[3],
            'product_test' : obj[4]
        }
        data_list.append(data_dic)

    conn.close()

    return data_list

In [4]:
quality_data = get_quality_data_list_for_predict()

In [5]:
quality_df = pd.DataFrame(quality_data)

In [6]:
quality_df.head()

,product_key,product_size_h,product_size_l,product_size_w,product_test
0,2020-10-31 16:38:15.951609-W1P0,60.0001,200.0010,99.9996,OK
1,2020-10-31 16:38:15.951609-body,49.9983,200.0010,99.9996,OK
2,2020-10-31 16:38:15.951609-wavyfin,60.0001,99.9994,49.9968,OK
3,2020-10-31 16:38:25.871779-W2P0,60.0001,200.0010,139.9980,OK
4,2020-10-31 16:38:25.871779-pipe1,30.0007,29.9990,49.9981,OK


In [7]:
quality_df.shape

(192326, 5)

In [8]:
quality_data

[{'product_key': '2020-10-31 16:38:15.951609-W1P0',
  'product_size_l': 200.001,
  'product_size_w': 99.9996,
  'product_size_h': 60.0001,
  'product_test': 'OK'},
 {'product_key': '2020-10-31 16:38:15.951609-body',
  'product_size_l': 200.001,
  'product_size_w': 99.9996,
  'product_size_h': 49.9983,
  'product_test': 'OK'},
 {'product_key': '2020-10-31 16:38:15.951609-wavyfin',
  'product_size_l': 99.9994,
  'product_size_w': 49.9968,
  'product_size_h': 60.0001,
  'product_test': 'OK'},
 {'product_key': '2020-10-31 16:38:25.871779-W2P0',
  'product_size_l': 200.001,
  'product_size_w': 139.998,
  'product_size_h': 60.0001,
  'product_test': 'OK'},
 {'product_key': '2020-10-31 16:38:25.871779-pipe1',
  'product_size_l': 29.999,
  'product_size_w': 49.9981,
  'product_size_h': 30.0007,
  'product_test': 'OK'},
 {'product_key': '2020-10-31 16:38:35.729839-W3P0',
  'product_size_l': 200.001,
  'product_size_w': 179.998,
  'product_size_h': 60.0001,
  'product_test': 'OK'},
 {'product_ke

## 설비 데이터 가져오기

In [9]:
def get_machine_data_list_for_predict():
    conn = pymysql.connect(host='127.0.0.1', user='root', password='carry789', db='projectdata', charset='utf8')

    sql = '''
        select product_key, machine_data, machine_data_code, process_time, end_time
        from machine
    '''

    cursor = conn.cursor()
    cursor.execute(sql)
    row = cursor.fetchall()

    data_list = []

    for obj in row:
        data_dic = {
            'product_key' : obj[0],
            'machine_data': obj[1],
            'machine_data_code': obj[2],
            'process_time': obj[3],
            'end_time': obj[4]
        }
        data_list.append(data_dic)

    conn.close()

    return data_list

In [10]:
machine_data = get_machine_data_list_for_predict()

In [11]:
machine_data

[{'product_key': '2020-10-31 16:38:15.951609-W1P0',
  'machine_data': 96.3835,
  'machine_data_code': 'E01',
  'process_time': 9.56103,
  'end_time': datetime.datetime(2020, 10, 31, 16, 38, 16)},
 {'product_key': '2020-10-31 16:38:25.871779-W2P0',
  'machine_data': 99.4246,
  'machine_data_code': 'E01',
  'process_time': 9.92017,
  'end_time': datetime.datetime(2020, 10, 31, 16, 38, 26)},
 {'product_key': '2020-10-31 16:38:35.729839-W3P0',
  'machine_data': 92.004,
  'machine_data_code': 'E01',
  'process_time': 9.85806,
  'end_time': datetime.datetime(2020, 10, 31, 16, 38, 36)},
 {'product_key': '2020-10-31 16:38:45.587019-W4P0',
  'machine_data': 506.854,
  'machine_data_code': 'T01',
  'process_time': 9.85718,
  'end_time': datetime.datetime(2020, 10, 31, 16, 38, 46)},
 {'product_key': '2020-10-31 16:38:54.696989-W5P0',
  'machine_data': 508.294,
  'machine_data_code': 'T01',
  'process_time': 9.10997,
  'end_time': datetime.datetime(2020, 10, 31, 16, 38, 55)},
 {'product_key': '202

In [12]:
machine_df = pd.DataFrame(machine_data)

In [16]:
machine_df.tail()

,end_time,machine_data,machine_data_code,process_time,product_key
95976,2020-11-25 14:45:08,509.8260,T01,9.53664,2020-11-25 14:45:08.352853-W5P12632
95977,2020-11-25 14:45:08,496.8810,T01,9.52124,2020-11-25 14:45:08.352853-W4P12633
95978,2020-11-25 14:45:08,98.5657,E01,9.27426,2020-11-25 14:45:08.352853-W3P12634
95979,2020-11-25 14:45:08,96.7370,E01,9.53493,2020-11-25 14:45:08.352853-W2P12635
95980,2020-11-25 14:45:08,98.5741,E01,9.86838,2020-11-25 14:45:08.352853-W1P12636


In [14]:
machine_df.shape

(95981, 5)

## 데이터프레임 만들기


In [15]:
product_key_list = []
body_l_list = []
body_w_list = []
body_h_list = []

wavyfin_l_list = []
wavyfin_w_list = []
wavyfin_h_list = []

op10_l_list = []
op10_w_list = []
op10_h_list = []

pipe1_l_list = []
pipe1_w_list = []
pipe1_h_list = []

op20_l_list = []
op20_w_list = []
op20_h_list = []

pipe2_l_list = []
pipe2_w_list = []
pipe2_h_list = []

op30_l_list = []
op30_w_list = []
op30_h_list = []

flange1_l_list = []
flange1_w_list = []
flange1_h_list = []

op40_l_list = []
op40_w_list = []
op40_h_list = []

flange2_l_list = []
flange2_w_list = []
flange2_h_list = []

op50_l_list = []
op50_w_list = []
op50_h_list = []

op60_l_list = []
op60_w_list = []
op60_h_list = []

op10_electricity_list = []
op10_process_time_list = []

op20_electricity_list = []
op20_process_time_list = []

op30_electricity_list = []
op30_process_time_list = []

op40_temperature_list = []
op40_process_time_list = []

op50_temperature_list = []
op50_process_time_list = []

op60_process_time_list = []

op10_test_list = []
op20_test_list = []
op30_test_list = []
op40_test_list = []
op50_test_list = []
op60_test_list = []

op10_timestamp = []
op20_timestamp = []
op30_timestamp = []
op40_timestamp = []
op50_timestamp = []
op60_timestamp = []


for instance in quality_data:
    
    if (instance['product_key'][27:29] == 'W6'):
        product_key_list.append(instance['product_key'])
        
    if (instance['product_key'][27:] == 'body'):
        body_l_list.append(instance['product_size_l'])
        body_w_list.append(instance['product_size_w'])
        body_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:] == 'wavyfin'):
        wavyfin_l_list.append(instance['product_size_l'])
        wavyfin_w_list.append(instance['product_size_w'])
        wavyfin_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W1'):
        op10_l_list.append(instance['product_size_l'])
        op10_w_list.append(instance['product_size_w'])
        op10_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W1'):
        if instance['product_test'] == 'OK':
            op10_test_list.append(0)
        else:
            op10_test_list.append(1)
        
    if (instance['product_key'][27:] == 'pipe1'):
        pipe1_l_list.append(instance['product_size_l'])
        pipe1_w_list.append(instance['product_size_w'])
        pipe1_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W2'):
        op20_l_list.append(instance['product_size_l'])
        op20_w_list.append(instance['product_size_w'])
        op20_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W2'):
        if instance['product_test'] == 'OK':
            op20_test_list.append(0)
        else:
            op20_test_list.append(1)
        
    if (instance['product_key'][27:] == 'pipe2'):
        pipe2_l_list.append(instance['product_size_l'])
        pipe2_w_list.append(instance['product_size_w'])
        pipe2_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W3'):
        op30_l_list.append(instance['product_size_l'])
        op30_w_list.append(instance['product_size_w'])
        op30_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W3'):
        if instance['product_test'] == 'OK':
            op30_test_list.append(0)
        else:
            op30_test_list.append(1)
        
    if (instance['product_key'][27:] == 'flange1'):
        flange1_l_list.append(instance['product_size_l'])
        flange1_w_list.append(instance['product_size_w'])
        flange1_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W4'):
        op40_l_list.append(instance['product_size_l'])
        op40_w_list.append(instance['product_size_w'])
        op40_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W4'):
        if instance['product_test'] == 'OK':
            op40_test_list.append(0)
        else:
            op40_test_list.append(1)
        
    if (instance['product_key'][27:] == 'flange2'):
        flange2_l_list.append(instance['product_size_l'])
        flange2_w_list.append(instance['product_size_w'])
        flange2_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W5'):
        op50_l_list.append(instance['product_size_l'])
        op50_w_list.append(instance['product_size_w'])
        op50_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W5'):
        if instance['product_test'] == 'OK':
            op50_test_list.append(0)
        else:
            op50_test_list.append(1)
        
    if (instance['product_key'][27:29] == 'W6'):
        op60_l_list.append(instance['product_size_l'])
        op60_w_list.append(instance['product_size_w'])
        op60_h_list.append(instance['product_size_h'])
        
    if (instance['product_key'][27:29] == 'W6'):
        if instance['product_test'] == 'OK':
            op60_test_list.append(0)
        else:
            op60_test_list.append(1)
        
for instance in machine_data:
    
    if (instance['product_key'][27:29] == 'W1'):
        op10_electricity_list.append(instance['machine_data'])
        
    if (instance['product_key'][27:29] == 'W1'):
        op10_process_time_list.append(instance['process_time'])
        
    if (instance['product_key'][27:29] == 'W1'):
        op10_timestamp.append(instance['end_time'])
        
    if (instance['product_key'][27:29] == 'W2'):
        op20_electricity_list.append(instance['machine_data'])
        
    if (instance['product_key'][27:29] == 'W2'):
        op20_process_time_list.append(instance['process_time'])
        
    if (instance['product_key'][27:29] == 'W2'):
        op20_timestamp.append(instance['end_time'])
        
    if (instance['product_key'][27:29] == 'W3'):
        op30_electricity_list.append(instance['machine_data'])
        
    if (instance['product_key'][27:29] == 'W3'):
        op30_process_time_list.append(instance['process_time'])
        
    if (instance['product_key'][27:29] == 'W3'):
        op30_timestamp.append(instance['end_time'])
        
    if (instance['product_key'][27:29] == 'W4'):
        op40_temperature_list.append(instance['machine_data'])
        
    if (instance['product_key'][27:29] == 'W4'):
        op40_process_time_list.append(instance['process_time'])
        
    if (instance['product_key'][27:29] == 'W4'):
        op40_timestamp.append(instance['end_time'])
        
    if (instance['product_key'][27:29] == 'W5'):
        op50_temperature_list.append(instance['machine_data'])
        
    if (instance['product_key'][27:29] == 'W5'):
        op50_process_time_list.append(instance['process_time'])
        
    if (instance['product_key'][27:29] == 'W5'):
        op50_timestamp.append(instance['end_time'])
        
    if (instance['product_key'][27:29] == 'W6'):
        op60_process_time_list.append(instance['process_time'])
        
    if (instance['product_key'][27:29] == 'W6'):
        op60_timestamp.append(instance['end_time'])
        
df = pd.DataFrame({"product_key" : product_key_list, 'body_l' : body_l_list, 'body_w' : body_w_list, 'body_h' : body_h_list,
                   'wavyfin_l' : wavyfin_l_list, 'wavyfin_w' : wavyfin_w_list, 'wavyfin_h' : wavyfin_h_list,
                   'op10_l' : op10_l_list, 'op10_w' : op10_w_list, 'op10_h' : op10_h_list,
                   'op10_electricity' : op10_electricity_list, 'op10_process_time' : op10_process_time_list,
                   'op10_test' : op10_test_list, 'op10_timestamp' : op10_timestamp,
                   
                   'pipe1_l' : pipe1_l_list, 'pipe1_w' : pipe1_w_list, 'pipe1_h' : pipe1_h_list,
                   'op20_l' : op20_l_list, 'op20_w' : op20_w_list, 'op20_h' : op20_h_list,
                   'op20_electricity' : op20_electricity_list, 'op20_process_time' : op20_process_time_list,
                   'op20_test' : op20_test_list, 'op20_timestamp' : op20_timestamp,
                   
                   'pipe2_l' : pipe2_l_list, 'pipe2_w' : pipe2_w_list, 'pipe2_h' : pipe2_h_list,
                   'op30_l' : op30_l_list, 'op30_w' : op30_w_list, 'op30_h' : op30_h_list,
                   'op30_electricity' : op30_electricity_list, 'op30_process_time' : op30_process_time_list,
                   'op30_test' : op30_test_list, 'op30_timestamp' : op30_timestamp,
                   
                   'flange1_l' : flange1_l_list, 'flange1_w' : flange1_w_list, 'flange1_h' : flange1_h_list,
                   'op40_l' : op40_l_list, 'op40_w' : op40_w_list, 'op40_h' : op40_h_list,
                   'op40_temperature' : op40_temperature_list, 'op40_process_time' : op40_process_time_list,
                   'op40_test' : op40_test_list, 'op40_timestamp' : op40_timestamp,
                   
                   'flange2_l' : flange2_l_list, 'flange2_w' : flange2_w_list, 'flange2_h' : flange2_h_list,
                   'op50_l' : op50_l_list, 'op50_w' : op50_w_list, 'op50_h' : op50_h_list,
                   'op50_temperature' : op50_temperature_list, 'op50_process_time' : op50_process_time_list,
                   'op50_test' : op50_test_list, 'op50_timestamp' : op50_timestamp,
                   
                   'op60_l' : op60_l_list, 'op60_w' : op60_w_list, 'op60_h' : op60_h_list,
                   'op60_process_time' : op60_process_time_list, 
                   'op60_test' : op60_test_list, 'op60_timestamp' : op60_timestamp})

ValueError: arrays must all be same length

In [ ]:
df.head()

## 부품 데이터로 양품 불량 예측 하기 (XGboost)

In [18]:
X = df.drop(['op60_timestamp', 'op60_process_time', 
             'op60_h', 'op60_w', 'op60_l', 'op60_test', 
             'op50_timestamp', 'op50_process_time', 
             'op50_h', 'op50_w', 'op50_l', 'op50_temperature', 'op50_test',
             'op40_timestamp', 'op40_process_time', 
             'op40_h', 'op40_w', 'op40_l', 'op40_temperature', 'op40_test',
             'flange1_l', 'flange1_w', 'flange1_h', 'flange2_l', 'flange2_w', 'flange2_h', 
             'op10_timestamp', 'op20_timestamp', 'op30_timestamp',
             'product_key'], axis = 1)
X.head()

,body_l,body_w,body_h,wavyfin_l,wavyfin_w,wavyfin_h,op10_l,op10_w,op10_h,op10_electricity,...,op20_test,pipe2_l,pipe2_w,pipe2_h,op30_l,op30_w,op30_h,op30_electricity,op30_process_time,op30_test
0,200.001,99.9996,49.9983,99.9994,49.9968,60.0001,200.001,99.9996,60.0001,96.3835,...,0,29.9949,50.0008,30.0014,200.001,179.998,60.0001,92.0040,9.85806,0
1,199.999,99.9967,50.0004,99.9994,50.0027,60.0057,199.999,99.9967,60.0057,96.1544,...,0,29.9978,49.9976,30.0030,199.999,179.995,60.0057,90.5035,9.77801,0
2,199.999,99.9991,50.0002,100.0040,50.0012,60.0035,199.999,99.9991,60.0035,89.4278,...,0,29.9971,49.9978,30.0020,199.999,179.994,60.0035,95.8751,9.80490,0
3,199.999,99.9965,49.9978,99.9996,50.0007,60.0007,199.999,99.9965,60.0007,91.6879,...,0,29.9989,50.0010,29.9991,199.999,179.997,60.0007,93.8166,9.77961,0
4,199.999,100.0050,50.0025,99.9983,50.0028,60.0012,199.999,100.0050,60.0012,90.5444,...,0,30.0022,49.9988,30.0051,199.999,180.008,60.0012,98.1582,9.99238,0


In [19]:
y = df['op60_test']
y.head()

0    0
1    0
2    0
3    0
4    0
Name: op60_test, dtype: int64

In [20]:
print(df['op60_test'].value_counts())

0    7909
1    2091
Name: op60_test, dtype: int64


In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size=0.1, 
                                                    shuffle=False, 
                                                    random_state=1004)

In [22]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9000, 30)
(1000, 30)
(9000,)
(1000,)


In [23]:
X_train.head()

,body_l,body_w,body_h,wavyfin_l,wavyfin_w,wavyfin_h,op10_l,op10_w,op10_h,op10_electricity,...,op20_test,pipe2_l,pipe2_w,pipe2_h,op30_l,op30_w,op30_h,op30_electricity,op30_process_time,op30_test
0,200.001,99.9996,49.9983,99.9994,49.9968,60.0001,200.001,99.9996,60.0001,96.3835,...,0,29.9949,50.0008,30.0014,200.001,179.998,60.0001,92.0040,9.85806,0
1,199.999,99.9967,50.0004,99.9994,50.0027,60.0057,199.999,99.9967,60.0057,96.1544,...,0,29.9978,49.9976,30.0030,199.999,179.995,60.0057,90.5035,9.77801,0
2,199.999,99.9991,50.0002,100.0040,50.0012,60.0035,199.999,99.9991,60.0035,89.4278,...,0,29.9971,49.9978,30.0020,199.999,179.994,60.0035,95.8751,9.80490,0
3,199.999,99.9965,49.9978,99.9996,50.0007,60.0007,199.999,99.9965,60.0007,91.6879,...,0,29.9989,50.0010,29.9991,199.999,179.997,60.0007,93.8166,9.77961,0
4,199.999,100.0050,50.0025,99.9983,50.0028,60.0012,199.999,100.0050,60.0012,90.5444,...,0,30.0022,49.9988,30.0051,199.999,180.008,60.0012,98.1582,9.99238,0


In [24]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 30 columns):
body_l               10000 non-null float64
body_w               10000 non-null float64
body_h               10000 non-null float64
wavyfin_l            10000 non-null float64
wavyfin_w            10000 non-null float64
wavyfin_h            10000 non-null float64
op10_l               10000 non-null float64
op10_w               10000 non-null float64
op10_h               10000 non-null float64
op10_electricity     10000 non-null float64
op10_process_time    10000 non-null float64
op10_test            10000 non-null int64
pipe1_l              10000 non-null float64
pipe1_w              10000 non-null float64
pipe1_h              10000 non-null float64
op20_l               10000 non-null float64
op20_w               10000 non-null float64
op20_h               10000 non-null float64
op20_electricity     10000 non-null float64
op20_process_time    10000 non-null float64
op20_test     

In [25]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: op60_test, dtype: int64

In [26]:
dtrain = xgb.DMatrix(data = X_train, label = y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)

In [27]:
wlist = [(dtrain, 'train'), (dtest, 'eval')]

In [28]:
params = {'max_depth' : 10,
         'eta': 0.01,
         'objective' : 'binary:logistic',
          'eval_metric' : 'error',
         'early_stopping':200
         }
num_rounds = 1000

In [29]:
xgb_model = xgb.train(params = params, dtrain = dtrain, num_boost_round = num_rounds,
                     early_stopping_rounds = 200, evals = wlist)

[0]	train-error:0.014889	eval-error:0.023
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 200 rounds.
[1]	train-error:0.013889	eval-error:0.019
[2]	train-error:0.014444	eval-error:0.018
[3]	train-error:0.014111	eval-error:0.019
[4]	train-error:0.014111	eval-error:0.019
[5]	train-error:0.014222	eval-error:0.018
[6]	train-error:0.014111	eval-error:0.019
[7]	train-error:0.013667	eval-error:0.019
[8]	train-error:0.014222	eval-error:0.019
[9]	train-error:0.014111	eval-error:0.019
[10]	train-error:0.014222	eval-error:0.019
[11]	train-error:0.014222	eval-error:0.019
[12]	train-error:0.014333	eval-error:0.019
[13]	train-error:0.014222	eval-error:0.019
[14]	train-error:0.014222	eval-error:0.019
[15]	train-error:0.014111	eval-error:0.019
[16]	train-error:0.014222	eval-error:0.019
[17]	train-error:0.014111	eval-error:0.019
[18]	train-error:0.014111	eval-error:0.019
[19]	train-error:0.014111	eval-error:0.019
[20]

[188]	train-error:0.011667	eval-error:0.016
[189]	train-error:0.011667	eval-error:0.016
[190]	train-error:0.011667	eval-error:0.016
[191]	train-error:0.011667	eval-error:0.016
[192]	train-error:0.011667	eval-error:0.016
[193]	train-error:0.011556	eval-error:0.016
[194]	train-error:0.011556	eval-error:0.016
[195]	train-error:0.011444	eval-error:0.016
[196]	train-error:0.011444	eval-error:0.016
[197]	train-error:0.011444	eval-error:0.016
[198]	train-error:0.011444	eval-error:0.016
[199]	train-error:0.011444	eval-error:0.016
[200]	train-error:0.011444	eval-error:0.016
[201]	train-error:0.011444	eval-error:0.016
[202]	train-error:0.011444	eval-error:0.016
[203]	train-error:0.011444	eval-error:0.016
[204]	train-error:0.011444	eval-error:0.016
[205]	train-error:0.011444	eval-error:0.016
[206]	train-error:0.011444	eval-error:0.016
[207]	train-error:0.011444	eval-error:0.016
[208]	train-error:0.011444	eval-error:0.016
[209]	train-error:0.011333	eval-error:0.016
[210]	train-error:0.011222	eval-

In [30]:
pred_probs = xgb_model.predict(dtest)
print(np.round(pred_probs[:10], 3))

print('predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시')
preds = [1 if x > 0.5 else 0 for x in pred_probs]
print('예측값 100개만 표시: ', preds[:100])

[0.029 0.024 0.027 0.021 0.017 0.017 0.019 0.03  0.031 0.024]
predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시
예측값 100개만 표시:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [31]:
from sklearn.metrics import accuracy_score, precision_score , recall_score , confusion_matrix

def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy , precision ,recall))

In [32]:
get_clf_eval(y_test, preds)

오차 행렬
[[792   0]
 [ 16 192]]
정확도: 0.9840, 정밀도: 1.0000, 재현율: 0.9231


In [ ]:
joblib.dump(rf_clf, 'trained_model_xgb.pkl')
print("파일 저장완료")